## Anna Jazayeri 131661209
### Workshop 5
### Text Mining BDM 550

In [1]:
import zipfile
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

import numpy as np
import re

from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import PlaintextCorpusReader

In [5]:
import zipfile
import os
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Download and unzip the labeled and unlabeled datasets
with zipfile.ZipFile('labeled_dataset.zip', 'r') as labeled_zip:
    labeled_zip.extractall('labeled_dataset')

with zipfile.ZipFile('unlabeled_dataset.zip', 'r') as unlabeled_zip:
    unlabeled_zip.extractall('unlabeled_dataset')

# Assigning the labeled and unlabeled datasets
label = 'labeled_dataset'
unlabel = 'unlabeled_dataset'

# Function to extract labels and texts from a file
def extract_labels_and_texts(file_path):
    labels = []
    texts = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Splitting each line into label and text
            parts = line.strip().split(maxsplit=1)
            if len(parts) == 2:
                label, text = parts
                labels.append(label)
                texts.append(text)
    return labels, texts

# Lists to store labels and texts
all_labels = []
all_texts = []

# Loop through labeled files to extract labels and texts
for file in os.listdir(label):
    if os.path.isfile(os.path.join(label, file)):
        labels, texts = extract_labels_and_texts(os.path.join(label, file))
        all_labels.extend(labels)
        all_texts.extend(texts)

# Tokenizing and cleaning the labeled texts
clean_label = [remove_stop_punc(text) for text in all_texts]

# Vectorizing with TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_label = tfidf_vectorizer.fit_transform(clean_label)

# Create a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Split the labeled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_label, all_labels, test_size=0.2, random_state=42)

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Predict labels for the unlabeled data
predicted_labels = nb_classifier.predict(tfidf_unlabel)

# Print the classification report
print(classification_report(y_test, nb_classifier.predict(X_test), target_names=categories))


NameError: name 'tfidf_unlabel' is not defined

In [2]:
import os
import zipfile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Download and unzip the labeled and unlabeled datasets
with zipfile.ZipFile('labeled_dataset.zip', 'r') as labeled_zip:
    labeled_zip.extractall('labeled_dataset')

with zipfile.ZipFile('unlabeled_dataset.zip', 'r') as unlabeled_zip:
    unlabeled_zip.extractall('unlabeled_dataset')

# Step 1: Load the labeled dataset
labeled_dataset_path = 'labeled_dataset'
unlabeled_dataset_path = 'unlabeled_dataset'

def remove_stop_punc(txt):
    # Tokenizing the text
    tokens = nltk.word_tokenize(txt)
    # removing the stopwords and punctuations
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
    # removing remaining punctuation using regex
    tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens]
    # now the cleaned tokens must be join together as a string
    clean_txt = ' '.join(tokens)
    return clean_txt

labeled_data = []
labels = []

for file in os.listdir(labeled_dataset_path):
    if os.path.isfile(os.path.join(labeled_dataset_path, file)):
        with open(os.path.join(labeled_dataset_path, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split(' ', 1)
                if len(parts) == 2:
                    label, sentence = parts
                    labeled_data.append(remove_stop_punc(sentence.strip()))
                    labels.append(label)
                    
X_train, X_test, y_train, y_test = train_test_split(labeled_data, labels, test_size=0.2, random_state=42)   
                    
# Feature extraction using CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

y_pred = classifier.predict(X_test_vectorized)
print("Classification Report:\n", classification_report(y_test, y_pred))

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

## <span style="color:blue;">Task 1: Preprocess the text to remove any stop words or punctuations.</span>

In [15]:
# Download and unzip the labeled and unlabeled datasets
with zipfile.ZipFile('labeled_dataset.zip', 'r') as labeled_zip:
    labeled_zip.extractall('labeled_dataset')

with zipfile.ZipFile('unlabeled_dataset.zip', 'r') as unlabeled_zip:
    unlabeled_zip.extractall('unlabeled_dataset')

# Step 1: Load the labeled dataset
labeled_dataset_path = 'labeled_dataset'
unlabeled_dataset_path = 'unlabeled_dataset'


In [6]:

labeled_data = []
labels = []

for file in os.listdir(labeled_dataset_path):
    if os.path.isfile(os.path.join(labeled_dataset_path, file)):
        with open(os.path.join(labeled_dataset_path, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split(' ', 1)
                if len(parts) == 2:
                    label, sentence = parts
                    labeled_data.append(sentence)
                    labels.append(label)
                    


In [7]:
labeled_data

['abstract ###',
 'the internet as level topology has been extensively studied over the past few years  little is known about the details of the as taxonomy',
 'as  node  can represent a wide variety of organizations  e g   large isp  or small private business  university  with vastly different network characteristics  external connectivity patterns  network growth tendencies  and other properties that we can hardly neglect while working on veracious internet representations in simulation environments',
 'this paper  we introduce a radically new approach based on machine learning techniques to map all the ases in the internet into a natural as taxonomy',
 'successfully classify  NUMBER   NUMBER  percent  of ases with expected accuracy of  NUMBER   NUMBER  percent',
 'release to the community the as level topology dataset augmented with   NUMBER   the as taxonomy information and  NUMBER   the set of as attributes we used to classify ases',
 'believe that this dataset will serve as an in

In [8]:
labels

['###',
 'MISC\talthough',
 'MISC\tan',
 'AIMX\tin',
 'OWNX\twe',
 'OWNX\twe',
 'OWNX\twe',
 '###',
 'MISC\tthe',
 'MISC\tfrom',
 'MISC\tthis',
 'MISC\tin',
 'MISC\tstatistical',
 'MISC\tin',
 'MISC\tfor',
 'MISC\tthe',
 'MISC\ton',
 'MISC\tsince',
 'MISC\tmoreover',
 'MISC\tfor',
 'MISC\ton',
 'MISC\tthus',
 'MISC\tan',
 'MISC\tfor',
 'MISC\tgiven',
 'AIMX\tin',
 'OWNX\twe',
 'OWNX\twe',
 'AIMX\tthen',
 'OWNX\twe',
 'OWNX\tour',
 'OWNX\tfinally',
 'OWNX\tin',
 'OWNX\tsection',
 'OWNX\tsection',
 'OWNX\twe',
 '###',
 'MISC\talthough',
 'MISC\tan',
 'AIMX\tin',
 'OWNX\twe',
 'OWNX\twe',
 'OWNX\twe',
 '###',
 'MISC\tthe',
 'MISC\tfrom',
 'MISC\tthis',
 'MISC\tin',
 'MISC\tstatistical',
 'MISC\tin',
 'MISC\tfor',
 'MISC\tthe',
 'MISC\ton',
 'MISC\tsince',
 'MISC\tmoreover',
 'MISC\tfor',
 'MISC\ton',
 'MISC\tthus',
 'MISC\tan',
 'MISC\tfor',
 'MISC\tgiven',
 'AIMX\tin',
 'AIMX\twe',
 'OWNX\twe',
 'OWNX\tthen',
 'OWNX\twe',
 'OWNX\tour',
 'OWNX\tfinally',
 'BASE\tin',
 'OWNX\tsection',
 'O

In [13]:
X_train, X_test, y_train, y_test = train_test_split(labeled_data, labels, test_size=0.2, random_state=42)   
                    
# Feature extraction using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

y_pred = classifier.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division= np.nan))

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [9]:
import os
import zipfile

labeled_zip_path = 'labeled_dataset.zip'
unlabeled_zip_path = 'unlabeled_dataset.zip'

if not os.path.exists(labeled_zip_path) or not os.path.exists(unlabeled_zip_path):
    print("Error: ZIP files not found.")
else:
    with zipfile.ZipFile(labeled_zip_path, 'r') as labeled_zip:
        labeled_zip.extractall('labeled_dataset')

    with zipfile.ZipFile(unlabeled_zip_path, 'r') as unlabeled_zip:
        unlabeled_zip.extractall('unlabeled_dataset')
labeled_data = [sentence for sentence in labeled_data if sentence.strip()]
from sklearn.metrics import confusion_matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


NameError: name 'y_test' is not defined

In [2]:
from pathlib import Path
import pandas as pd

# Path to the folder containing the .txt files
folder_path = Path("C:\\Users\\annaj\\OneDrive\\Desktop\\Git Projects\\Text-Classification\\labeled_dataset")

# Get a list of all .txt files in the folder
txt_files = folder_path.glob('*.txt')

# Create a list to store the content of each file
file_contents = []

# Loop through each .txt file and append its content to the list
for txt_file in txt_files:
    with open(txt_file, 'r') as file:
        content = file.read()
        file_contents.append(content)

# Concatenate the content using pandas
concatenated_content = pd.concat([pd.Series(file_contents)], ignore_index=True)

# Save the concatenated content to a new file
output_file_path = "C:\\Users\\annaj\\OneDrive\\Desktop\\Git Projects\\Text-Classification\\la.txt"
concatenated_content.to_csv(output_file_path, index=False, header=False)


In [5]:
labeled_data

['abstract',
 'internet level topology extensively studied past years little known details taxonomy',
 'node represent wide variety organizations e g large isp small private business university vastly different network characteristics external connectivity patterns network growth tendencies properties hardly neglect working veracious internet representations simulation environments',
 'paper introduce radically new approach based machine learning techniques map ases internet natural taxonomy',
 'successfully classify number number percent ases expected accuracy number number percent',
 'release community level topology dataset augmented number taxonomy information number set attributes used classify ases',
 'believe dataset serve invaluable addition understanding structure evolution internet',
 'introduction',
 'rapid expansion internet last two decades produced large scale system thousands diverse independently managed networks collectively provide global connectivity across wide spec

In [4]:
# Use the trained classifier to predict labels for unlabeled data
unlabeled_data = []

for file in os.listdir(unlabeled_dataset_path):
    if os.path.isfile(os.path.join(unlabeled_dataset_path, file)):
        with open(os.path.join(unlabeled_dataset_path, file), 'r', encoding='utf-8') as f:
            unlabeled_data.extend(remove_stop_punc(line.strip()) for line in f.readlines())

# Vectorize the unlabeled data
unlabeled_data_vectorized = vectorizer.transform(unlabeled_data)

# Predict labels for the unlabeled data
predicted_labels = classifier.predict(unlabeled_data_vectorized)

# Print or save the predicted labels for further use
for sentence, label in zip(unlabeled_data, predicted_labels):
    print(f"{label}: {sentence}")



###: abstract
OWNX: transporter analyses conducted 141 organisms whose complete genome sequences available
OWNX	we: organism complete set membrane transport systems identified predicted functions classified protein families based transporter classification system
OWNX	we: organisms larger genome sizes generally possessed relatively greater number transport systems
MISC: prokaryotes unicellular eukaryotes significant factor increase transporter content genome size greater diversity transporter types
OWNX	we: contrast multicellular eukaryotes greater number paralogs specific transporter families important factor increase transporter content genome size
MISC: eukaryotic prokaryotic intracellular pathogens endosymbionts exhibited markedly limited transport capabilities
OWNX	we: hierarchical clustering phylogenetic profiles transporter families derived presence absence certain transporter family showed clustering patterns organisms correlated evolutionary history overall physiology lifestyl

## <span style="color:blue;">Task 2: Use TF-IDF to vectorize the sentences.</span>

## <span style="color:blue;">Task 3: Use Scikit learn to create a (naïve bayes) classifier.</span>

<span style="color:blue;">**First create a classifier (naïve bayes) to classify the given dataset into 6 categories: (AIMX, OWNX, CONT, BASE, NUMBER, and MISC).<br>Then, use the classifier to label the sentences in the unlabeled dataset.**</span>

## <span style="color:blue;">Task 4: Summarize your work and your findings in a few sentences.</span>

### <span style="color:blue;">BONUS [Extra 5% added to any workshops]: Use other classification approaches to label the unlabeled sentences. Evaluate your work using precision, recall, and f1 score.</span>